## Evaluating regression techniques for speaker characterization
### Laura Fernández Gallardo

Similarly as done for classification, the performances of different regression techniques for characterizing users by their voices are now assessed.

I am addressing the prediction of each of the 34 interpersonal speaker [characteristics](https://github.com/laufergall/Subjective_Speaker_Characteristics) (continuous numeric labels of the [NSC corpus](http://www.qu.tu-berlin.de/?id=nsc-corpus)). These characteristics are: 

'non_likable', 'secure', 'attractive', 'unsympathetic', 'indecisive', 'unobtrusive', 'distant', 'bored', 'emotional', 'not_irritated', 'active', 'pleasant', 'characterless', 'sociable', 'relaxed', 'affectionate', 'dominant', 'unaffected', 'hearty', 'old', 'personal', 'calm', 'incompetent', 'ugly', 'friendly', 'masculine', 'submissive', 'indifferent', 'interesting', 'cynical', 'artificial', 'intelligent', 'childish', 'modest'.

I also address the prediction of the 5 traits: 'warmth', 'attractiveness', 'confidence', 'compliance', and 'maturity'. These were obtained after factor analysis on the 34-dimensional ratings of speaker characteristics.

I will consider the common RMSE (root mean squared error) and the more robust to outliers MAPE (median absolute percentage error) as the metrics for success:

\begin{equation}
RMSE = \sqrt{\frac{\sum_i(y_i-\hat{y}_i)^2}{n}}  
\end{equation}

\begin{equation}
MAPE = median(\left | \frac{y_i-\hat{y}_i}{y_i} \right |)
\end{equation}

where $y_i$ and $\hat{y_i}$ are the observed and the predicted values for the $i^{th}$ data point.

In this notebook, I will prepare data for the evaluation of regression: 
* speech features (eGeMAPS)
* questionnaire item ratings of speaker characteristics (x34, from 0 to 100)
* trait scores of speaker characteristics (x5, normalized)

In [1]:
import io
import requests

import numpy as np
import pandas as pd

%matplotlib inline

In [2]:
# fix random seed for reproducibility
seed = 2302
np.random.seed(seed)

In [3]:
# names of items and traits

items_names = ['non_likable', 'secure', 'attractive', 'unsympathetic', 'indecisive', 'unobtrusive', 'distant', 'bored', 'emotional', 'not_irritated', 'active', 'pleasant', 'characterless', 'sociable', 'relaxed', 'affectionate', 'dominant', 'unaffected', 'hearty', 'old', 'personal', 'calm', 'incompetent', 'ugly', 'friendly', 'masculine', 'submissive', 'indifferent', 'interesting', 'cynical', 'artificial', 'intelligent', 'childish', 'modest']

traits_names = ['warmth', 'attractiveness', 'compliance', 'confidence', 'maturity']

## Speaker characteristics: item ratings

The files "SC_ratings_medians.csv" and "SC_ratings_medians.csv" have been generated in the ..\data folder.

We consider the same train/test partition of speakers as done for the WAAT classification task.

In [4]:
# load scores (averaged across listeners)

path = "https://raw.githubusercontent.com/laufergall/ML_Speaker_Characteristics/master/data/generated_data/"

url = path + "ratings_SC_means.csv"
s = requests.get(url).content
ratings =pd.read_csv(io.StringIO(s.decode('utf-8')))

ratings.head()

,speaker_ID,speaker_gender,non_likable,secure,attractive,unsympathetic,indecisive,unobtrusive,distant,bored,...,friendly,masculine,submissive,indifferent,interesting,cynical,artificial,intelligent,childish,modest
0,1,female,36.571429,65.214286,59.785714,37.357143,33.714286,66.857143,35.642857,35.642857,...,75.428571,20.285714,59.000000,34.571429,60.571429,43.071429,35.785714,65.285714,46.857143,61.071429
1,2,female,66.666667,57.200000,39.333333,54.066667,33.066667,57.266667,56.466667,55.733333,...,55.600000,18.333333,55.133333,58.733333,38.533333,51.533333,63.200000,51.133333,33.533333,60.266667
2,3,female,45.812500,72.562500,47.125000,30.937500,27.937500,46.250000,38.625000,33.437500,...,64.125000,19.937500,46.437500,41.562500,55.562500,50.250000,40.687500,60.250000,14.437500,54.812500
3,4,male,40.071429,59.857143,44.571429,54.428571,35.071429,52.285714,48.571429,49.785714,...,51.428571,75.785714,47.071429,51.357143,49.142857,55.857143,38.071429,55.785714,40.500000,46.928571
4,5,male,42.117647,60.529412,53.823529,50.764706,35.705882,59.764706,49.764706,42.647059,...,54.176471,80.764706,47.823529,53.235294,57.352941,47.705882,35.823529,62.823529,29.294118,49.823529


## Speaker traits: scores

Scaled scores (with mean = 0 and std = 1) of speakers on these dimensions were already extracted in the [subjective analysis](https://github.com/laufergall/Subjective_Speaker_Characteristics), for males and for females separately.

In [5]:
# speaker scores

path = "https://raw.githubusercontent.com/laufergall/Subjective_Speaker_Characteristics/master/data/generated_data/"

url = path + "factorscores_malespk.csv"
s = requests.get(url).content
scores_m =pd.read_csv(io.StringIO(s.decode('utf-8')))

url = path + "factorscores_femalespk.csv"
s = requests.get(url).content
scores_f =pd.read_csv(io.StringIO(s.decode('utf-8')))

# rename dimensions
scores_m.columns = ['sample_heard', 'warmth', 'attractiveness', 'confidence', 'compliance', 'maturity']
scores_f.columns = ['sample_heard', 'warmth', 'attractiveness', 'compliance', 'confidence', 'maturity']

# join male and feame scores
scores = scores_m.append(scores_f)
scores['gender'] = scores['sample_heard'].str.slice(0,1)
scores['spkID'] = scores['sample_heard'].str.slice(1,4).astype('int')

scores.head()

,attractiveness,compliance,confidence,maturity,sample_heard,warmth,gender,spkID
0,-0.579301,-0.921918,0.608503,0.276580,m004_linden_stimulus.wav,-0.284638,m,4
1,0.442865,-0.950212,0.588889,0.630295,m005_nicosia_stimulus.wav,-0.494019,m,5
2,-0.507534,0.139302,-0.151077,-0.669449,m006_rabat_stimulus.wav,1.533478,m,6
3,1.180748,-0.108982,0.962166,1.026359,m007_klaksvik_stimulus.wav,0.478983,m,7
4,1.070247,-0.284278,-0.875589,-1.291311,m016_beirut_stimulus.wav,1.861551,m,16


## Train/test partitions and merge

In [6]:
# read partitions from multioutput multiclass classification
# (stratified taking into account speaker gender)

path2 = 'https://raw.githubusercontent.com/laufergall/ML_Speaker_Characteristics/master/data/generated_data/'

url = path2 + "classes_train.csv"
s = requests.get(url).content
classes_train =pd.read_csv(io.StringIO(s.decode('utf-8')))

url = path2 + "classes_test.csv"
s = requests.get(url).content
classes_test =pd.read_csv(io.StringIO(s.decode('utf-8')))

# partitions of ratings and scores
ratings = ratings.rename(index=str, columns={'speaker_ID': 'spkID'})
ratings_train = ratings[ratings['spkID'].isin(classes_train['spkID'])] # shape (225, 36)
ratings_test = ratings[ratings['spkID'].isin(classes_test['spkID'])] # shape (75, 36)

ratings_scores_train = ratings_train.merge(scores.drop(['sample_heard','gender'],axis=1)) # shape (225, 41)
ratings_scores_test = ratings_test.merge(scores.drop(['sample_heard','gender'],axis=1)) # shape (75, 41)


## Speech features

Same features as for classification (of clean speech).

In [7]:
path3 = "https://raw.githubusercontent.com/laufergall/ML_Speaker_Characteristics/master/data/extracted_features/"

url = path3 + "/eGeMAPSv01a_semispontaneous_splitted.csv"
s = requests.get(url).content
feats =pd.read_csv(io.StringIO(s.decode('utf-8')), sep = ';') # shape: 3591, 89

# extract speaker ID from speech file name
feats['spkID'] = feats['name'].str.slice(2, 5).astype('int')

# appending multilabels to features
feats_ratings_scores_train = pd.merge(feats, ratings_scores_train) # shape (2700, 130)
feats_ratings_scores_test = pd.merge(feats, ratings_scores_test) # shape (891, 130)

In [8]:
# save feature names
dropcolumns = ['name','spkID','speaker_gender'] + items_names + traits_names
feats_names = list(feats_ratings_scores_train.drop(dropcolumns, axis=1))

myfile = open(r'..\data\generated_data\feats_names.txt', 'w')
for item in feats_names:
    myfile.write("%r\n" % item)
    
# save names of speaker items and traits
    
myfile = open(r'..\data\generated_data\items_names.txt', 'w')
for item in items_names:
    myfile.write("%r\n" % item)
    
myfile = open(r'..\data\generated_data\traits_names.txt', 'w')
for item in traits_names:
    myfile.write("%r\n" % item)

In [9]:
# 'name' + 'speaker_gender' + 'spkID' + 88 features + 5 trait scores + 34 item ratings

list(feats_ratings_scores_test)

['name',
 'F0semitoneFrom27.5Hz_sma3nz_amean',
 'F0semitoneFrom27.5Hz_sma3nz_stddevNorm',
 'F0semitoneFrom27.5Hz_sma3nz_percentile20.0',
 'F0semitoneFrom27.5Hz_sma3nz_percentile50.0',
 'F0semitoneFrom27.5Hz_sma3nz_percentile80.0',
 'F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2',
 'F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope',
 'F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope',
 'loudness_sma3_amean',
 'loudness_sma3_stddevNorm',
 'loudness_sma3_percentile20.0',
 'loudness_sma3_percentile50.0',
 'loudness_sma3_percentile80.0',
 'loudness_sma3_pctlrange0-2',
 'loudness_sma3_meanRisingSlope',
 'loudness_sma3_stddevRisingSlope',
 'loudness_sma3_meanFallingSlope',
 'loudness_sma3_stddevFallingSlope',
 'spectralFlux_sma3_amean',
 'spectralFlux_sma3_stddevNorm',
 'mfcc1_sma3_amean',
 'mfcc1_sma3_stddevNorm',
 'mfcc2_sma3_amean',
 'mfcc2_sma3_stddevNorm',
 'mfcc3_sma3_amean',
 'mfcc3_sma3_stddevNorm',

Save to csv to use in other notebooks.

In [10]:
feats_ratings_scores_train.to_csv(r'..\data\generated_data\feats_ratings_scores_train.csv', index=False)
feats_ratings_scores_test.to_csv(r'..\data\generated_data\feats_ratings_scores_test.csv', index=False)